<a href="https://colab.research.google.com/github/charlessantarosa/dl-emotion-recognition-uea/blob/master/dl_emotion_recognition_uea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
    <h1><strong>Dataset: Natural Human Face Images for Emotion Recognition</strong></h1>
    <h4>Link: <a href="https://www.kaggle.com/sudarshanvaidya/random-images-for-face-emotion-recognition">https://www.kaggle.com/sudarshanvaidya/random-images-for-face-emotion-recognition</a></h4>
    <div id="text">
        <p>
São 5.500 + imagens com 8 categorias de emoções - raiva, desprezo, nojo, medo, felicidade, neutralidade, tristeza e surpresa. Todas as imagens contêm rosto humano em tons de cinza (ou esboço). Cada imagem tem uma escala de cinza de 224 x 224 pixels no formato PNG.</p>
<p>As imagens são obtidas na Internet, onde estão disponíveis gratuitamente para download. Google, Unsplash, Flickr etc.</p>
    </div>
</div>

## Configuração e Bibliotecas

In [ ]:
import os
import cv2
from urllib.request import urlopen
import math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras.preprocessing.image import img_to_array
from tensorflow.keras import layers

# from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
# from skimage import transform

from PIL import Image
import numpy as np



In [ ]:
# Currenty Directory
PATH_ROOT = !pwd
PATH_ROOT

# Global Vars
DATASET_FILE = 'dl-emotion-recognition-uea.zip'
DATASET_GOOGLE_DRIVER_ID = '1J0xMHoQqC6zBibSxukK-Gshfb5kK-P1y'
DATASET_PATH = PATH_ROOT[0] + '/dataset/'
DATASET_PREDICT = PATH_ROOT[0] + '/predict'

In [ ]:
# Download Dataset.zip from GoogleDriver
!gdown --id $DATASET_GOOGLE_DRIVER_ID
!unzip -o $DATASET_FILE > /dev/null

# !ls -la $DATASET_PATH
# DATASET_PATH = '/home/charles/uea/deep-learning/datasets/dl-emotion-recognition-uea/'
# !ls -la $DATASET_PATH

print('\nLista de pastas ....')
!ls -la $DATASET_PATH

## Processando Dataset

In [ ]:
# Processing directory dataset

total_images = 0
classes = []
for _dir in os.listdir(DATASET_PATH):
    count = 0
    for f in os.listdir(DATASET_PATH + _dir + "/"):
        count += 1
        total_images += 1
        
    classes.append(_dir)
    print(f"{_dir} => {count} imagens")
    
print('\n----------------------------------------------')    
print(f"\nTotal: {total_images} imagens")
print('Total de classes: ', len(classes))
print('Classes: ', classes)
print('\n----------------------------------------------')    

In [ ]:
img_arr = np.empty(shape=(total_images, 48, 48, 3)) # Dimensão da imagem 48,48
img_label = np.empty(shape=(total_images))
label_to_text = {}

i = 0
e = 0
for dir_ in os.listdir(DATASET_PATH):
    if dir_ in classes:
        label_to_text[e] = dir_
        for f in os.listdir(DATASET_PATH + dir_ + "/"):

            image = cv2.imread(DATASET_PATH + dir_ + "/" + f)
            img_arr[i] = cv2.resize(image, (48, 48))


            img_label[i] = e

            i += 1
        print(f"Imagens {dir_} carregadas!!!")
        e += 1

img_arr.shape, img_label

In [ ]:
img_arr

In [ ]:
# Classes

label_to_text

## Visualização dos dados

In [ ]:
# Visualização Randômica de 2 imagens por classe.

fig = pyplot.figure(1, (8, 8))

idx = 0
for k in label_to_text:
    
    sample_indices = np.random.choice(np.where(img_label==k)[0], size=2, replace=False)
    sample_images = img_arr[sample_indices]
    
    for img in sample_images:
        idx += 1
        ax = pyplot.subplot(4, 4, idx)
        ax.imshow(img[:,:,0], cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(label_to_text[k])
        pyplot.tight_layout()


## Trainamento

In [ ]:
img_arr = img_arr.reshape(len(img_arr), 48, 48, 3)
data = np.array(img_arr, dtype="float32") / 255.0
# test_data = np.array(test_data, dtype="float") / 255.0

le = LabelEncoder()
train_label = le.fit_transform(img_label)
train_label = to_categorical(train_label)

X_train, X_test, y_train, y_test = train_test_split(data, train_label, test_size=0.1, random_state=42)
print(X_train, X_test, y_train, y_test)

In [ ]:
model = Sequential()
num_classes = y_train.shape[1]

model.add(Conv2D(500, kernel_size=(3,3), input_shape=(48, 48, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(50, activation='relu'))
model.add(Dropout(0.2))

# Predição
model.add(Dense(num_classes, activation='softmax', name='predict'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model.summary()

In [ ]:
# Fit
history = model.fit(x=X_train, y=y_train,  epochs=15, verbose=1, validation_data=(X_test,y_test))

In [ ]:
# Acurária
scores = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss: %.6f" % (scores[0]))
print("Test Acurária: %.2f%%" % (scores[1]*100))

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test Loss: %.6f" % (scores[0]))
print('Test accuracy:', score[1])

In [ ]:
ig, ax = plt.subplots(1,2, figsize=(16,8))
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

## Validaçao do modelo


In [ ]:
!rm -rf $DATASET_PREDICT
!git clone https://github.com/charlessantarosa/dl-emotion-recognition-uea.git $DATASET_PREDICT
!ls -la $DATASET_PREDICT

In [ ]:
def getImage(filename):
  
  img_path = DATASET_PREDICT + '/' + filename
  image_display = Image.open(img_path)
  image_size= (48,48)

  img = keras.preprocessing.image.load_img(img_path, target_size=image_size)
  np_image = keras.preprocessing.image.img_to_array(img)
  np_image = tf.expand_dims(np_image, 0)  # Create batch axis

  return np_image, image_display

In [ ]:
print('\nEmotions Recognition ...\n')
label_to_text

In [ ]:
img_pred, image = getImage('e1.png')
plt.imshow(image)

predictions = model.predict(img_pred)
score = predictions.argmax()

print(f'Resultado: [{score}] => ' + label_to_text[score])

In [ ]:
img_pred, image = getImage('e2.png')
plt.imshow(image)

predictions = model.predict(img_pred)
score = predictions.argmax()

print(f'Resultado: [{score}] => ' + label_to_text[score])

In [ ]:
img_pred, image = getImage('e3.png')
plt.imshow(image)

predictions = model.predict(img_pred)
score = predictions.argmax()

print(f'Resultado: [{score}] => ' + label_to_text[score])

In [ ]:
img_pred, image = getImage('e4.png')
plt.imshow(image)

predictions = model.predict(img_pred)
score = predictions.argmax()

print(f'Resultado: [{score}] => ' + label_to_text[score])

In [ ]:
img_pred, image = getImage('e5.png')
plt.imshow(image)

predictions = model.predict(img_pred)
score = predictions.argmax()

print(f'Resultado: [{score}] => ' + label_to_text[score])

In [ ]:
img_pred, image = getImage('e6.png')
plt.imshow(image)

predictions = model.predict(img_pred)
score = predictions.argmax()

print(f'Resultado: [{score}] => ' + label_to_text[score])